<a href="https://colab.research.google.com/github/Zia-Ur-Rehman1/Machine-Learning/blob/master/Machine_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip "/content/drive/My Drive/ML/Data.zip"


Archive:  /content/drive/My Drive/ML/Data.zip
replace Data/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [10]:
import numpy as np
import pickle
import weakref
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [11]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((100, 100))
directory_root = '/content/Data'
width=100
height=100
depth=3


In [12]:
image_list, label_list = [], []
img_size=100
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
      fp = os.path.join(directory_root,directory)
      for eachImage in os.listdir(fp):
        imagePath = os.path.join(fp,eachImage)
        img = cv2.imread(imagePath)
        resized = cv2.resize(img , (img_size , img_size))
        image_list.append(resized)
        label_list.append(directory)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Image loading completed


In [13]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [14]:
np_image_list = np.array(image_list) / 255.0
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))

[INFO] Spliting data to train, test


In [15]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (height, width, depth)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("sigmoid"))
# model.summary()

In [16]:
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [17]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(np_image_list.shape,image_labels.shape)


(1980, 100, 100, 3) (1980, 1)
(495, 100, 100, 3) (495, 1)
(2475, 100, 100, 3) (2475, 1)


In [18]:
history = model.fit(
    np_image_list, image_labels, 32,
    validation_split=0.2,    
    epochs=EPOCHS, verbose=1
    )

Epoch 1/10
62/62 [==============================] - 42s 44ms/step - loss: 0.8664 - accuracy: 0.7679 - val_loss: 12.0562 - val_accuracy: 0.0000e+00
Epoch 2/10
62/62 [==============================] - 1s 21ms/step - loss: 0.1553 - accuracy: 0.9499 - val_loss: 4.9061 - val_accuracy: 0.0000e+00
Epoch 3/10
62/62 [==============================] - 1s 20ms/step - loss: 0.0977 - accuracy: 0.9689 - val_loss: 7.7075 - val_accuracy: 0.0000e+00
Epoch 4/10
62/62 [==============================] - 1s 21ms/step - loss: 0.0986 - accuracy: 0.9644 - val_loss: 17.5215 - val_accuracy: 0.0000e+00
Epoch 5/10
62/62 [==============================] - 1s 20ms/step - loss: 0.0711 - accuracy: 0.9791 - val_loss: 6.0879 - val_accuracy: 0.1838
Epoch 6/10
62/62 [==============================] - 1s 20ms/step - loss: 0.1215 - accuracy: 0.9577 - val_loss: 1.4738 - val_accuracy: 0.4586
Epoch 7/10
62/62 [==============================] - 1s 19ms/step - loss: 0.0420 - accuracy: 0.9888 - val_loss: 1.8393 - val_accuracy: 0